# Create config files and Creating Glue database

## Create a config file

In [ ]:
%%writefile ../src/config.py

PYTHON_GENERATION_MODEL_ID = "us.amazon.nova-pro-v1:0"
SQL_GENERATION_MODEL_ID = "us.amazon.nova-pro-v1:0"
AWS_REGION = "us-west-2"
DATABASE_NAME = "insurance_and_eclaims"
BUCKET_NAME = "<Your-bucket-name>"
ATHENA_S3_OUTPUT = f"s3://{BUCKET_NAME}/athena_out/"
ATHENA_WORKGROUP = "primary"

## Import

In [ ]:
from datetime import date
import pandas as pd
from collections import Counter
from collections import defaultdict
import sys
sys.path.append("../src/")
from config import *


# User Inputs

In [ ]:
sqlite_dir = os.path.join("..", "data", "spider_data", "database", "insurance_and_eClaims")
csv_dir = os.path.join("..", "data", "spider_data_csv")

BUCKET_PATH = f's3://{BUCKET_NAME}/spider_data_csv/'
DATABASE_NAME = 'insurance_and_eclaims'
CRAWLER_NAME = 'insurance_and_eclaims_crawler'

# Create Glue Role

In [ ]:
import boto3
import json

def create_glue_crawler_role(role_name, bucket_name):
    iam = boto3.client('iam')

    # Define the trust relationship policy
    trust_relationship = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "glue.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    # Create the IAM role
    try:
        response = iam.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(trust_relationship)
        )
        print(f"Role {role_name} created successfully.")
    except iam.exceptions.EntityAlreadyExistsException:
        print(f"Role {role_name} already exists.")

    # Attach the AWS Glue Service Role policy
    iam.attach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/service-role/AWSGlueServiceRole'
    )
    print("AWSGlueServiceRole policy attached.")

    # Define the S3 bucket access policy
    s3_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "s3:GetObject",
                    "s3:ListBucket"
                ],
                "Resource": [
                    f"arn:aws:s3:::{bucket_name}",
                    f"arn:aws:s3:::{bucket_name}/*"
                ]
            }
        ]
    }

    # Create and attach the S3 bucket access policy
    iam.put_role_policy(
        RoleName=role_name,
        PolicyName=f"GlueCrawlerS3Policy-{bucket_name}",
        PolicyDocument=json.dumps(s3_policy)
    )
    print(f"S3 bucket access policy for {bucket_name} attached.")

    # Get and return the role ARN
    role = iam.get_role(RoleName=role_name)
    return role['Role']['Arn']

# Usage
role_name = "GlueCrawlerRole-NovaBlog"

IAM_ROLE_ARN = create_glue_crawler_role(role_name, BUCKET_NAME)
print(f"Glue Crawler Role ARN: {IAM_ROLE_ARN}")


# Utils

In [ ]:
def sanitize_name(name):
    return ''.join(c if c.isalnum() else '_' for c in name).rstrip('_')

def sqlite_to_csv(sqlite_file, csv_base_dir):
    db_name = sanitize_name(os.path.splitext(os.path.basename(sqlite_file))[0])
    
    # Connect to the SQLite database
    conn = sqlite3.connect(sqlite_file)
    
    # Create a cursor
    cursor = conn.cursor()
    
    # Get list of tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    
    for table_name in tables:
        table_name = table_name[0]
        try:
            # Read the table into a pandas DataFrame
            df = pd.read_sql_query(f"SELECT * FROM '{table_name}'", conn)
            
            # Add columns for metadata
            df['source_database'] = db_name
            df['source_table'] = table_name
            
            # Create the output directory
            output_dir = os.path.join(csv_base_dir, db_name, sanitize_name(table_name))
            os.makedirs(output_dir, exist_ok=True)
            
            # Write the DataFrame to a CSV file
            csv_file = os.path.join(output_dir, f"{sanitize_name(table_name)}.csv")
            df.to_csv(csv_file, index=False, encoding='utf-8-sig')
            print(f"Successfully converted table: {table_name}")
        except Exception as e:
            print(f"Error processing table {table_name}: {str(e)}")
            print(f"Skipping table {table_name} and moving to the next one.")
            continue
    
    # Close the connection
    conn.close()

## Convert to CSV

In [ ]:
processed_folders = []
not_processed_folders = []
problematic_databases = []

# Create the base CSV directory if it doesn't exist
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)

# Iterate through the folders
for root, dirs, files in os.walk(sqlite_dir):
    sqlite_files = [file for file in files if file.endswith('.sqlite')]
    
    if sqlite_files:
        processed_folders.append(os.path.relpath(root, sqlite_dir))
        
        for file in sqlite_files:
            sqlite_path = os.path.join(root, file)
            
            print(f"\nConverting {sqlite_path} to CSV...")
            try:
                sqlite_to_csv(sqlite_path, csv_dir)
                print(f"Conversion complete for {sqlite_path}")
            except Exception as e:
                print(f"Error processing database {sqlite_path}: {str(e)}")
                problematic_databases.append(f"{sqlite_path}: {str(e)}")
                print(f"Skipping database {sqlite_path} and moving to the next one.")
                continue
    elif os.path.isdir(os.path.join(sqlite_dir, root)) and not any(file.endswith('.sqlite') for file in files):
        not_processed_folders.append(os.path.relpath(root, sqlite_dir))

print("All conversions completed. Summary written to 'conversion_summary.txt'.")
print(f"CSV files are stored in the '{csv_dir}' directory.")

# S3 sync

In [ ]:
! aws s3 cp {csv_dir} s3://{BUCKET_NAME}/spider_data_csv/ --recursive

# Glue database and Crawler

In [ ]:
import boto3
import time

def create_glue_database_and_crawler(bucket_path, database_name, crawler_name, iam_role_arn):
    """
    Creates a Glue crawler to catalog S3 data and create a database
    
    Parameters:
    bucket_path (str): S3 path (e.g., 's3://my-bucket/data/')
    database_name (str): Name for the Glue database
    crawler_name (str): Name for the Glue crawler
    iam_role_arn (str): IAM role ARN with necessary permissions
    """
    
    # Initialize Glue client
    glue_client = boto3.client('glue')
    
    try:
        # Create Glue Database
        print(f"Creating database: {database_name}")
        glue_client.create_database(
            DatabaseInput={
                'Name': database_name,
                'Description': f'Database created for {bucket_path}'
            }
        )
        
        # Create Glue Crawler
        print(f"Creating crawler: {crawler_name}")
        glue_client.create_crawler(
            Name=crawler_name,
            Role=iam_role_arn,
            DatabaseName=database_name,
            Targets={
                'S3Targets': [
                    {
                        'Path': bucket_path
                    }
                ]
            },
            Description=f'Crawler for {bucket_path}',
            SchemaChangePolicy={
                'UpdateBehavior': 'UPDATE_IN_DATABASE',
                'DeleteBehavior': 'LOG'
            }
        )
        
        # Start the crawler
        print("Starting crawler...")
        glue_client.start_crawler(Name=crawler_name)
        
        # Wait for crawler to complete
        while True:
            crawler_state = glue_client.get_crawler(Name=crawler_name)['Crawler']['State']
            if crawler_state == 'READY':
                break
            print(f"Crawler state: {crawler_state}")
            time.sleep(30)
        
        print("Crawler completed successfully!")
        
    except Exception as e:
        print(f"Error: {str(e)}")
        raise

In [ ]:
create_glue_database_and_crawler(
    bucket_path=BUCKET_PATH,
    database_name=DATABASE_NAME,
    crawler_name=CRAWLER_NAME,
    iam_role_arn=IAM_ROLE_ARN
)